<a href="https://colab.research.google.com/github/resh1998/VoterCRM_Backend/blob/main/BackEnd/Data_Collection/Copy_of_Clean_csv_data_Updated_Kurnool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Steps to Run the File

1. Make sure all the files converted from PDF to CSV are in Raw_CSV_files
2. Mount the drive (commented as Step 2)
3. Create CSV_files folder. If the folder already exists then output message would be "Cannot create directory" (commented as Step 3)
4. Run the remaining cells one after the other


#### Mounting Drive

In [ ]:
# Step 2
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Step 3
!mkdir /content/drive/MyDrive/CSV_files

mkdir: cannot create directory ‘/content/drive/MyDrive/CSV_files’: File exists


#### Text column Split into diff columns

In [ ]:
#df['Age'] = df['text'].str.extract(r'((?<=Age)(.|\s)*(?=Gender))')

In [ ]:
def process_text_col(df):

  df['text'] = df.text.str.replace('\n', ' ')

  df['text'] = df.text.str.replace('Name ', 'Name')
  df['text'] = df.text.str.replace('Gender ', 'Gender')
  df['text'] = df.text.str.replace('Age ', 'Age')
  df['text'] = df.text.str.replace('House Number ', 'House Number')

  df['Age'] = df['text'].str.extract(r'((?<=Age).*(?=Gender))')
  df['Gender'] = df['text'].str.extract(r'((?<=Gender).*(?=))')

  def condition(x):
    if 'Husband' in x:
      return '2'
    elif 'Wife' in x:
      return '4'
    elif 'Father' in x:
      return '1'
    elif 'Mother' in x:
      return '3'
    elif 'Guru' in x:
      return '6'
    else:
      return '5'

  df['Relation_type'] = df['text'].apply(condition)

  df['House_Number'] = df['text'].str.extract(r'((?<=Number).*(?=Age))')

  df['text'] = df.text.str.replace('Husband', 'Relation')
  df['text'] = df.text.str.replace('Wife', 'Relation')
  df['text'] = df.text.str.replace('Father', 'Relation')
  df['text'] = df.text.str.replace('Mother', 'Relation')
  df['text'] = df.text.str.replace('Other', 'Relation')
  df['text'] = df.text.str.replace('Guru', 'Relation')
  df['text'] = df.text.str.replace('Legal Guardian', 'Relation')
  df['text'] = df.text.str.replace('\'s', '')

  df['Name'] = df['text'].str.extract(r'((?<=Name).*(?=House))')
  df['Name'] = df['Name'].str.split('Relation').str[0]

  df['text'] = df['text'].str.split('House').str[0]
  df['text'] = df['text'].str.split('Relation').str[1]

  df['text'] = df.text.str.replace('Name', '')

  df.rename(columns= {'text': 'Relation_Name'}, inplace = True)

  df['Name'] = df['Name'].str.replace('!', 'I')
  df['Name'] = df['Name'].str.replace('\W', ' ', regex = True)
  df['Name'] = df.Name.str.strip()

  df['Relation_Name'] = df['Relation_Name'].str.replace('!', 'I')
  df['Relation_Name'] = df['Relation_Name'].str.replace('\W', ' ', regex = True)
  df['Relation_Name'] = df.Relation_Name.str.strip()

  df['House_Number'] = df.House_Number.str.strip()
  df['House_Number'] = df.House_Number.str.replace(r'^\W*', ' ', regex = True)

  df['Age'] = df.Age.str.replace('\W', '', regex = True)
  df['Gender'] = df.Gender.str.replace('\W', '', regex = True)

  return df

#### Voter columns Preprocessing

In [ ]:
import math
def process_voter_id_col(df):

  df['voter_id'] = df.voter_id.str.replace(r'((\n).*)', ' ', regex = True)
  df['voter_id'] = df.voter_id.str.replace('\n', '', regex = True)
  df['voter_id'] = df.voter_id.str.replace('\W', '', regex = True)

  df['voter_id_str_part'] = df.voter_id.str.findall(r'([a-zA-Z]+)')
  df['voter_id_str_part'] = df.voter_id_str_part.str.join(',').astype(str)

  df['voter_id_str_part_get_unique'] = df.voter_id.str.findall(r'([a-zA-Z]+)').str[0]
  list_str = list(df['voter_id_str_part_get_unique'].unique())

  list_str_n = list()

  for i in list_str:
    if i is not None and len(str(i)) >= 3 and str(i) != 'nan':
      i = str(i)[:3]
      list_str_n.append(str(i))

  list_str_n = list(np.unique(np.array(list_str_n)))

  df['voter_id_no_part'] = df.voter_id.str.findall(r'(\d+)')
  df['voter_id_no_part'] = df.voter_id_no_part.str.join(',').astype(str)
  df['voter_id_no_part'] = df['voter_id_no_part'].str.replace(',', '')

  def add_zeros(x):
    if x is not None and len(x) < 7:
      return x.zfill(7)
    elif x is not None and len(x) > 7:
      return x[-7:]
    else:
      return x

  df['voter_id_no_part'] = df['voter_id_no_part'].apply(add_zeros)

  def new_voter_id_col(num_part, str_part):
    #global list_str_n
    for i in list_str_n:
      if str_part is not None and str_part != '' and (i in str_part or i[:2] in str_part or i[::2] in str_part or i[1:] in str_part):
          return i + str(num_part)
      if str_part == '' or str_part is None:
        return None


  df['voter_id_new'] = df.apply(lambda x: new_voter_id_col(x['voter_id_no_part'], x['voter_id_str_part']), axis = 1)
  df['voter_id_new'] = df.voter_id_new.str.strip()


  df.drop(['voter_id_no_part', 'voter_id_str_part', 'voter_id'], axis = 1, inplace = True)

  return df

#  def voter_id_str_slice(x):
#    if x != '' and len(str(x)) > 3:
#      return x[:3]
#    else:
#      return x
#
#  df['voter_id_str_part'] = df['voter_id_str_part'].apply(voter_id_str_slice)

#### Header Section

In [ ]:
import re

def process_header_col(df):

  df['header_section'] = df['header_section'].str.replace('/n', ' ')

  df['Assembly_Constituency_No_and_Name'] = df['header_section'].str.extract(r'((?<=Assembly Constituency No and Name).*(?=Part))')
  df['Assembly_Constituency_No_and_Name'] = df['Assembly_Constituency_No_and_Name'].str.replace(':', '', regex = True)
  df['Assembly_Constituency_No_and_Name'] = df['Assembly_Constituency_No_and_Name'].str.strip()

  df['Section_No_and_Name'] = df['header_section'].str.extract(r'((?<=Section No and Name).*(?=))')
  df['Section_No_and_Name'] = df['Section_No_and_Name'].str.replace(': ', '')
  df['Section_No_and_Name'] = df['Section_No_and_Name'].str.strip()

  df['header_section'] = df['header_section'].str.replace('\n', ' ')

  df['Part_Number'] = df['header_section'].str.extract(r'((?<=Part number).*(?=Section))')

  df['Part_Number'] = df['Part_Number'].str.replace(': ', '').str.strip()

  df['Key'] = df['Assembly_Constituency_No_and_Name']
  df['Key'] = df['Key'].str.replace('\d', '', regex = True)
  df['Key'] = df['Key'].str.replace('\W', '', regex = True)

  df['Constituency'] = df['Assembly_Constituency_No_and_Name']
  df['Constituency'] = df['Constituency'].str.replace('\d', '', regex = True)
  df['Constituency'] = df['Constituency'].str.replace('\W', '', regex = True)

  def add_const_and_part(constituency_name, part_no):
    return str(constituency_name) + '-' + str(part_no)

  df['Key'] = df.apply(lambda x: add_const_and_part(x['Key'], x['Part_Number']), axis = 1)

  return df

#### Dataframe Editing

In [ ]:
import uuid

def edit_df(df):

  df = df.drop(['header_section'], axis = 1)
  df = df.rename(columns = {'voter_id_new': 'Voter_id', 'voter_no': 'Voter_no'})

  cols =  ['Voter_no', 'Voter_id', 'Name', 'Relation_Name', 'Relation_type', 'Age',
         'Gender', 'House_Number', 'Assembly_Constituency_No_and_Name',
          'Section_No_and_Name', 'Part_Number', 'Constituency', 'Key']

  df = df[cols]



  return df

#  def fill_null_voter_ids(x):
#    if x is None:
#      x = str(uuid.uuid4())[:9]
#      return '-' + x
#    else:
#      return x

#  df['Voter_id'] = df['Voter_id'].apply(fill_null_voter_ids)

#  df = df[df['Voter_id'].isnull() | ~ df[df['Voter_id'].notnull()].duplicated(subset = 'Voter_id', keep = 'first')]

#  df.drop_duplicates(keep = 'first', inplace = True)



#### Main Function

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path

csv_files = Path("/content/drive/MyDrive/Raw_CSV_Files_all/Kanigiri_Raw_CSV_Files").glob("*.csv")

for csv in csv_files:

  filename = csv.stem

  csv_path = '/content/drive/MyDrive/Raw_CSV_Files_all/Kanigiri_Raw_CSV_Files/' + filename + '.csv'

  df = pd.read_csv(csv_path)
  df = process_text_col(df)
  df = process_voter_id_col(df)
  df = process_header_col(df)
  df = edit_df(df)

  csv_path = '/content/drive/MyDrive/Raw_CSV_Files_all/Kanigiri_Clean_CSV_files/' + filename + '.csv'
  df.to_csv(csv_path)


#### Merging the Files and removing dups

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
csv_files = Path("/content/drive/MyDrive/Raw_CSV_Files_all/Kanigiri_Clean_CSV_files").glob("*.csv")


cols =  ['Voter_no', 'Voter_id', 'Name', 'Relation_Name', 'Relation_type', 'Age',
        'Gender', 'House_Number', 'Assembly_Constituency_No_and_Name',
        'Section_No_and_Name', 'Part_Number', 'Constituency', 'Key']

df_merged = pd.DataFrame(columns = cols)

for csv in csv_files:

  filename = csv.stem
  csv_path = '/content/drive/MyDrive/Raw_CSV_Files_all/Kanigiri_Clean_CSV_files/' + filename + '.csv'
  df = pd.read_csv(csv_path)
  df_merged = pd.concat([df_merged, df], ignore_index=True, sort=False)

csv_path = '/content/drive/MyDrive/Raw_CSV_Files_all/Kanigiri_Clean_CSV_files/Merged_Kanigiri_CSV_file.csv'

df_merged.to_csv(csv_path)




In [ ]:
df_merged.drop(['Unnamed: 0', 'Voter_no'], axis = 1, inplace =True)

df1 = df_merged[df_merged['Voter_id'].isnull()].drop_duplicates(keep = 'first')
df2 = df_merged[df_merged['Voter_id'].notnull()].drop_duplicates(subset = 'Voter_id', keep = 'first')

df_merged = pd.concat([df1, df2], ignore_index=True, sort=False)

df_merged['Relation_Name'] = np.where(df_merged['Relation_Name'].isnull() & (df_merged['Relation_type'] == 5), 'NA', df_merged['Relation_Name'])



csv_path = '/content/drive/MyDrive/Raw_CSV_Files_all/Kanigiri_Clean_CSV_files/Merged_Kanigiri_CSV_file_noDups.csv'

df_merged.to_csv(csv_path)